

# The PageRank Algorithm

*Note*: This set of notes is currently a stub. We'll add more content soon. 

As an example of running the PageRank algorithm, let's consider the network of mentions in the musical Hamilton. [We retrieved this data from the [public GitHub repository](https://github.com/ahunker/Hamilton-Project) of the [Hamilton Project](https://newtfire.org/hamilton/index.html).]{.aside}

![](https://cms-assets.soraschools.com/cdn-cgi/image/f=auto,w=2000/uploads/Sora_Journal_November_Thumbnails_1_e8f0d16610_a3c0784672.webp)

In this network, there is a directed arrow from character $A$ to character $B$ if character $A$ mentions character $B$ in a song. You might reasonably expect that the characters who are frequently mentioned in songs would tend to be the most important characters in the musical. 


In [ ]:
import networkx as nx
import numpy as np
import pandas as pd

df = pd.read_csv("https://philchodrow.github.io/PIC16A/homework/HW3-hamilton-data.csv", 
names = ["mentioner", "mentioned"])

df = df[df["mentioner"] != df["mentioned"]]

G = nx.from_pandas_edgelist(df, 
                            source = "mentioner", 
                            target = "mentioned", 
                            edge_attr=None, 
                            create_using=nx.DiGraph())

for i in range(2):
    G = G.subgraph([name for name, val in G.out_degree() if val > 0])

In [ ]:
layout = nx.spring_layout(G)

nx.draw(G, layout, 
            node_size = 500, 
            with_labels = True,
            edge_color = "lightgray", 
            node_color = "lavender",
            edgecolors  = "darkgray")

Now let's implement the PageRank algorithm for this graph. 

:::{.callout-note}
:::{#def-pagerank-walk}

## PageRank Random Walk

The **PageRank random walk** has two parameters: 

- The *teleportation vector* $\mathbf{v} \in \mathbb{R}^n_+$, which we assume to satisfy $\sum_{i\in N}v_i = 1$. 
- The *teleportation rate* $\alpha\in [0,1]$. 

This walk has transition probabilities 
$$
\mathbb{P}(X_{t+1} = i|X_{t}=j) = (1-\alpha)\frac{a_{ij}}{k_j^{\mathrm{out}}} + \alpha v_i\;. 
$${#eq-pagerank-transition}

Its transition matrix is 

$$
\mathbf{P} = (1-\alpha) \mathbf{A}(\mathbf{K}^{\mathrm{out}})^{-1} + \alpha \mathbf{V}\;,
$$

$$
\mathbf{V} = \left[\begin{matrix}
    | & | & \cdots & | \\ 
    \mathbf{v} & \mathbf{v} & \cdots & \mathbf{v} \\ 
    | & | & \cdots & |
\end{matrix}\right]\;.
$$

:::
:::

Here's the intuitive way to think about this walk. At each time step, the walker flips a weighted coin with probability of heads equal to $\alpha$. 

- If heads, the walker chooses to sample from the probability distribution encoded by $\mathbf{v}$. That is, the walker chooses from among all the nodes in $N$, and picks node $i$ with probability $v_i$. 
- If tails, the walker instead follows a link, just like in the directed random walk. 

This is why there are two terms in the transition probability in @eq-pagerank-transition. The first term corresponds to the "tails" scenario in which the walker does a step corresponding to the directed random walk, while the second term corresponds to teleportation. 

The standard choice of the teleportation vector is $\mathbf{v} = \frac{1}{n}\mathbf{1}$, so each node has an equal probability of being chosen for teleportation. However, it's also possible to take other approaches, as we'll see in a moment. 


## Implementation


In [ ]:
def pagerank(G, v, alpha = 0.15):

    assert np.isclose(v.sum(), 1.0)

    # construct the PR transition matrix
    A  = nx.to_numpy_array(G).T
    K  = np.diag(A.sum(axis = 0))     # K^out
    P_ = A@np.linalg.inv(K)           # random walk matrix
    V  = np.outer(v, np.ones(len(v))) # teleportation matrix
    P = (1-alpha)*P_ + alpha*V        # overall transition matrix

    # grab the eigenvector with eigenvalue 1, normalize it and return. 
    eigs = np.linalg.eig(P)        
    pi = eigs[1][:,np.isclose(eigs[0], 1.0)]
    pi = np.abs(pi) / np.abs(pi).sum()
    return pi

Let's try using PageRank with a uniform teleportation vector:  

In [ ]:
n = len(G.nodes())
v = 1/n*np.ones(n)
pagerank(G, v)

We can also redraw the graph with nodes scaled by their PageRank scores. 

In [ ]:
def draw_pagerank(G, v,  layout, alpha = 0.15):

    pi = pagerank(G, v, alpha = alpha)
    nx.draw(G, layout, 
            node_size = 5000*pi, 
            with_labels = True,
            edge_color = "lightgray", 
            node_color = "lavender",
            edgecolors  = "darkgray")

v = 1/n*np.ones(n)
draw_pagerank(G, v, layout, alpha = 0.1)

In [ ]:
v = np.array([1 if i == "kingGeorge" else 0 for i in list(G.nodes())])
draw_pagerank(G, v, layout)